Calculation of rewards for the main branch. Currently all our mechs have the same main branch

In [41]:
import numpy as np
from auto_robot_design.generator.restricted_generator.two_link_generator import TwoLinkGenerator
from auto_robot_design.pinokla.calc_criterion import (ActuatedMass, ImfCompute, ManipCompute,
                                                      MovmentSurface, NeutralPoseMass, TranslationErrorMSE, EffectiveInertiaCompute, ManipJacobian)
from auto_robot_design.pinokla.criterion_agregator import CriteriaAggregator
from auto_robot_design.pinokla.criterion_math import ImfProjections
from auto_robot_design.pinokla.default_traj import (convert_x_y_to_6d_traj_xz,
                                                    get_simple_spline, get_vertical_trajectory, create_simple_step_trajectory)
from auto_robot_design.optimization.rewards.reward_base import PositioningReward
from auto_robot_design.optimization.rewards.jacobian_and_inertia_rewards import HeavyLiftingReward, AccelerationCapability
from auto_robot_design.optimization.rewards.pure_jacobian_rewards import EndPointZRRReward, VelocityReward, ForceEllipsoidReward
from auto_robot_design.optimization.rewards.inertia_rewards import EndPointIMFReward, MassReward

from auto_robot_design.description.actuators import TMotor_AK10_9, TMotor_AK60_6, TMotor_AK70_10, TMotor_AK80_64, TMotor_AK80_9
from auto_robot_design.description.builder import ParametrizedBuilder, DetailedURDFCreatorFixedEE
from auto_robot_design.description.builder import jps_graph2pinocchio_robot

#### Setup trajectories 

In [42]:
ground_symmetric_step = convert_x_y_to_6d_traj_xz(*create_simple_step_trajectory(starting_point=[-0.5, -1.02], step_height= 0.4, step_width=1,n_points=50))
left_shift_step = convert_x_y_to_6d_traj_xz(*create_simple_step_trajectory(starting_point=[-0.8, -1.02], step_height= 0.4, step_width=1,n_points=50))
#right_shift_step = convert_x_y_to_6d_traj_xz(*create_simple_step_trajectory(starting_point=[-0.2, -1.02], step_height= 0.4, step_width=1,n_points=50))


central_vertical = convert_x_y_to_6d_traj_xz(*get_vertical_trajectory(-1.1, 0.6, 0,50))
left_vertical = convert_x_y_to_6d_traj_xz(*get_vertical_trajectory(-1.1, 0.6, -0.2,50))
#right_vertical = convert_x_y_to_6d_traj_xz(*get_vertical_trajectory(-1.1, 0.6, 0.2,50))

#### Characteristics to be calculated

In [43]:
# criteria that either calculated without any reference to points, or calculated through
# the aggregation of values from all points on trajectory
dict_trajectory_criteria = {
    "MASS": NeutralPoseMass(),
    "POS_ERR": TranslationErrorMSE()  # MSE of deviation from the trajectory
}
# criteria calculated for each point on the trajectory
dict_point_criteria = {
    # Impact mitigation factor along the axis
    "IMF": ImfCompute(ImfProjections.Z),
    "MANIP": ManipCompute(MovmentSurface.XZ),
    "Effective_Inertia": EffectiveInertiaCompute(),
    "Actuated_Mass": ActuatedMass(),
    "Manip_Jacobian": ManipJacobian(MovmentSurface.XZ)
}

crag = CriteriaAggregator(dict_point_criteria, dict_trajectory_criteria)

#### set the rewards and weights

In [44]:
rewards_step = [(PositioningReward(pos_error_key="POS_ERR"), 1), (AccelerationCapability(manipulability_key='Manip_Jacobian',
            trajectory_key="traj_6d", error_key="error", actuated_mass_key="Actuated_Mass"), 1), (EndPointIMFReward(imf_key='IMF',
            trajectory_key="traj_6d", error_key="error"), 1)]

#step_trajectories = [ground_symmetric_step, left_shift_step, right_shift_step]
step_trajectories = [ground_symmetric_step]
rewards_vertical = [(PositioningReward(pos_error_key="POS_ERR"), 1),
           (HeavyLiftingReward(manipulability_key='Manip_Jacobian',
            trajectory_key="traj_6d", error_key="error", mass_key="MASS"), 1)]
#vertical_trajectories = [central_vertical, left_vertical, right_vertical]
vertical_trajectories = [central_vertical, left_vertical]
# rewards = [(PositioningReward(pos_error_key="POS_ERR"), 1),
#            (HeavyLiftingReward(manipulability_key='Manip_Jacobian',
#             trajectory_key="traj_6d", error_key="error", mass_key="MASS"), 1),
#            (AccelerationCapability(manipulability_key='Manip_Jacobian',
#             trajectory_key="traj_6d", error_key="error", actuated_mass_key="Actuated_Mass"), 1),
#            (EndPointZRRReward(manipulability_key='Manip_Jacobian',
#             trajectory_key="traj_6d", error_key="error"), 1),
#            (VelocityReward(manipulability_key='Manip_Jacobian',
#             trajectory_key="traj_6d", error_key="error"), 1),
#            (EndPointIMFReward(imf_key='IMF',
#             trajectory_key="traj_6d", error_key="error"), 1),
#            (ForceEllipsoidReward(manipulability_key='Manip_Jacobian',
#             trajectory_key="traj_6d", error_key="error"), 1),
#            (MassReward(mass_key="MASS"), 1)
#            ]

In [45]:
actuator_list = [TMotor_AK10_9(), TMotor_AK60_6(), TMotor_AK70_10(), TMotor_AK80_64(), TMotor_AK80_9()]
generator = TwoLinkGenerator()
generator.build_standard_two_linker()
graph = generator.graph
# open chain is fully actuated
for jp in graph.nodes:
    if not jp.attach_endeffector:
        jp.active = True

rewards_and_trajectories = [(rewards_step, step_trajectories),(rewards_vertical, vertical_trajectories)] 


In [46]:
result_vector = {}
for j in actuator_list:
    # create builder
    thickness = 0.04
    builder = ParametrizedBuilder(DetailedURDFCreatorFixedEE, size_ground=np.array(
        [thickness*5, thickness*10, thickness*2]), actuator=j, thickness=thickness)

    fixed_robot, free_robot = jps_graph2pinocchio_robot(graph, builder)

    crag = CriteriaAggregator(
        dict_point_criteria, dict_trajectory_criteria)

    total_reward=0
    partial_rewards = []
    for rewards, trajectories in rewards_and_trajectories:
        max_reward = -float('inf')
        max_partial = None
        best_trajectory = None
        for trajectory_id, trajectory in enumerate(trajectories):
            point_criteria_vector, trajectory_criteria, res_dict_fixed = crag.get_criteria_data(fixed_robot, free_robot, trajectory)
            current_total = 0
            current_partial = []
            for reward, weight in rewards:
                current_partial.append(reward.calculate(point_criteria_vector, trajectory_criteria, res_dict_fixed, Actuator=j)[0])
                current_total += weight*current_partial[-1]
            if current_total > max_reward:
                max_reward = current_total
                max_partial = current_partial
                best_trajectory_id = trajectory_id
        total_reward+= max_reward
        partial_rewards.append((best_trajectory_id, max_partial))

    result_vector[j.__class__.__name__] = total_reward

print(result_vector)

{'TMotor_AK10_9': 48.00715940450545, 'TMotor_AK60_6': 11.012434154257727, 'TMotor_AK70_10': 28.545807631876844, 'TMotor_AK80_64': 83.40555452569498, 'TMotor_AK80_9': 20.371855871804183}
